## Timeseries Modeling - DeepState
### Objective:
The objective of the notebook is to -
* Backtest on all hyperparameters of DeepState provided in the config
* Find the best set of hyperparameters using the metric provided in the config

In [ ]:
import yaml
import inspect
import glob
import numpy as np
import pandas as pd
from distutils.command.config import config
from tqdm.auto import tqdm
from datetime import timedelta
from datetime import datetime
import mlflow
from sklearn.metrics import mean_absolute_error,mean_squared_error
import os
import dotsi
import logging

import mxnet as mx
# from gluonts.model import deepstate
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepStateEstimator
from gluonts.mx import Trainer
from gluonts.evaluation import make_evaluation_predictions, Evaluator

/databricks/python/lib/python3.8/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
 warnings.warn(
/databricks/python/lib/python3.8/site-packages/gluonts/model/deepstate/__init__.py:18: FutureWarning: The module gluonts.model.deepstate has been moved to gluonts.mx.model.deepstate. In GluonTS v0.12 it will be no longer possible to use the old path. Try to use 'from gluonts.mx import DeepStateEstimator'.
 warnings.warn(

In [ ]:
# logging part
p_dir = "/tmp/"
log_file = "DeepState_hyperparameter_tuning" + " (" +datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+ ").log"

# DeepState - Hyperparameter tuning logs
logger = logging.getLogger('custom_log')
logger.setLevel(logging.DEBUG)

# Applying necessary formatter
fh = logging.FileHandler(p_dir+log_file)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
fh.setFormatter(formatter)   
logger.addHandler(fh)

In [ ]:
# Getting the default settings of hyperparameters. Used to check that user-provided hyperparameters must always be a subset of these.
def get_default_args(func) -> dict:
    """Function to get the default values of the hyperparameters for the given algorithm

    Parameters
    ----------
    func : constructor of the respective algorithm
        The name of the algorithm (Eg: Prophet,SARIMAX)

    Returns
    -------
    dict
        returns a dictionary of hyperparameters and the corresponding default values for the given algorithm
    """
    
    signature = inspect.signature(func)
    return {
        k: v.default if v.default is not inspect.Parameter.empty else None
        for k, v in signature.parameters.items()
        if k != 'self'
    }

#### Configurable Hyperparameters
The following are the possible hyperparameters that can be tuned for DeepState. The preferred hyperparameters, and their respective search spaces, over which tuning is to be done need to be mentioned in the config file.

In [ ]:
default_hpps_init = get_default_args(DeepStateEstimator)
default_hpps_fit = get_default_args(Trainer)

if('trainer' in default_hpps_init.keys()):
    del default_hpps_init['trainer']
print("Hyperparameters under fit\n",default_hpps_fit,"\n\nHyperparameters under init\n",default_hpps_init)

Hyperparameters under fit
 {'ctx': None, 'epochs': 100, 'num_batches_per_epoch': 50, 'learning_rate': 0.001, 'clip_gradient': 10.0, 'weight_decay': 1e-08, 'init': 'xavier', 'hybridize': True, 'callbacks': None, 'add_default_callbacks': True} 

Hyperparameters under init
 {'freq': None, 'prediction_length': None, 'cardinality': None, 'add_trend': False, 'past_length': None, 'num_periods_to_train': 4, 'num_layers': 2, 'num_cells': 40, 'cell_type': 'lstm', 'num_parallel_samples': 100, 'dropout_rate': 0.1, 'use_feat_dynamic_real': False, 'use_feat_static_cat': True, 'embedding_dimension': None, 'issm': None, 'scaling': True, 'time_features': None, 'noise_std_bounds': gluonts.mx.distribution.lds.ParameterBounds(lower=1e-06, upper=1.0), 'prior_cov_bounds': gluonts.mx.distribution.lds.ParameterBounds(lower=1e-06, upper=1.0), 'innovation_bounds': gluonts.mx.distribution.lds.ParameterBounds(lower=1e-06, upper=0.01), 'batch_size': 32}

#### Processing Config file
Dependent variable, date variable, modeling granularity & other related modeling details are provided in the form of a config file. Each TS Algorithm, and the related hyperparameter values to be tried, should be given in the config file.

In [ ]:
%run ../../../0_Config.ipynb

In [ ]:
logger.info("Config file read")
assert set(app_config["Algorithms"]["DeepState"]["Hyperparameters"]['Estimator'].keys()).\
           issubset(set(default_hpps_init.keys())),\
           'keys supplied by the user for the DeepState Algorithm under __init__ method must be valid'
assert set(app_config["Algorithms"]["DeepState"]["Hyperparameters"]['Trainer'].keys()).\
           issubset(set(default_hpps_fit.keys())),\
           'keys supplied by the user for the DeepState Algorithm under fit method must be valid'

# For exporting the config file
temp_config = app_config.copy()

In [ ]:
def frange(start,stop,step= 1):
    l = []
    i = start
    while(i < stop):
        l.append(round(i,len(str(step))))
        i = i+step
    return l

def drange(hyperparameters):
    l=[]
    for key in hyperparameters.keys():
        val = hyperparameters[key]
        if 'range' in val:
            val = val.replace('range','frange')
            new_str = 'total_list = '  + val
            _locals = locals()
            exec(new_str,globals(),_locals)
            without_dup = list(set(_locals['total_list']))
            hyperparameters[key] = without_dup
    return hyperparameters

In [ ]:
fit_ = drange(app_config['Algorithms']['DeepState']['Hyperparameters']['Trainer'])
init_ = drange(app_config['Algorithms']['DeepState']['Hyperparameters']['Estimator'])
for key in init_.keys():
    if(key in fit_.keys()):
        fit_[key] = list(set(fit_[key]+init_[key]))
    else:
        fit_[key] = list(init_[key])
    
fit_new = {}
for key in fit_.keys():
    temp = []
    for val in fit_[key]:
        if(type(val) == list):
            val = str(val)
        if((val!='None') and (val!='Null') and (val!=None)):
            temp.append(val)
    if(len(temp)>0):
        fit_new[key] = temp
    
for val in ['prediction_length','freq','use_feat_static_real','use_feat_static_cat','cardinality']:
    if(val in fit_new.keys()):
        del fit_new[val]
        
if(len(app_config['Algorithms']['DeepState']['exogenous_variables']['feat_dynamic_real'])==0):
    fit_new['use_feat_dynamic_real'] = [False]
else:
    fit_new['use_feat_dynamic_real'] = [True]
    
app_config['Algorithms']['DeepState']['Hyperparameters'] = fit_new

In [ ]:
# Create the algo directory for storing the results
output_directory = app_config['output_dir_path']
root_dir = "Modeling_Results"
algorithm = "DeepState"
algo_path = os.path.join(output_directory,root_dir,algorithm)
if not os.path.exists(algo_path):
    os.makedirs(algo_path)
logger.info("Created algorithm directory")    

logs_path = os.path.join(output_directory,root_dir,'logs',algorithm)
if not os.path.exists(logs_path):
    os.makedirs(logs_path)
logger.info("Created logs directory")

config_path = os.path.join(app_config['output_dir_path'],"Modeling_Results","config")
if not os.path.exists(config_path):
    os.makedirs(config_path)
logger.info("Created config directory")

#### Broadcasting the required variables
Variables suffixed with "_conf" are taken from the config file

In [ ]:
hyperparameters_conf = dict(app_config["Algorithms"]["DeepState"]["Hyperparameters"])
# print(hyperparameters_conf)

modeling_granularity_conf = app_config["modeling_granularity"]
global_modeling_granularity_conf = app_config["Algorithms"]["DeepState"]["global_model_gran"]

# print(modeling_granularity_conf)

# Rename Start date and DV config
dv_config = app_config["dependent_variable"]
ds_config = app_config["date_var"]

# sales variable broadcast
if(app_config["Algorithms"]["DeepState"]["sales_amt_variable"]==app_config["dependent_variable"]):
    app_config["Algorithms"]["DeepState"]["sales_amt_variable"] = 'y'
broadcast_sales = dotsi.Dict({"value":app_config["Algorithms"]["DeepState"]["sales_amt_variable"]})

# pos and neg corr broadcast
corr_config = dict(app_config['Algorithms']['DeepState']['exogenous_variables'])
corr_config_broadcast = dotsi.Dict({"value":corr_config})

# Eval metric broadcast
broadcast_metric = dotsi.Dict({"value":app_config["validation"]["metric"]})
broadcast_test_periods = dotsi.Dict({"value":app_config["validation"]["no_of_backtesting_test_periods"]})
broadcast_tracking = dotsi.Dict({"value":app_config['tracking']})

broadcast_granularity = dotsi.Dict({"value":modeling_granularity_conf})
global_broadcast_granularity = dotsi.Dict({"value":global_modeling_granularity_conf})
broadcast_hyper_parameters = dotsi.Dict({"value":hyperparameters_conf})

logger.info("Broadcasted the required variables")

#### Pandas UDF for Backtesting
The UDF gets executed in multiple worker nodes to parallelize the backtesting process. All the broadcasted variables are accessed within the UDF as and when required

In [ ]:
def get_prediction_UDF(df_data: pd.DataFrame)-> pd.DataFrame:
    """Function to perform backtesting for the given input data using the broadcasted information from the config file

      Parameters
      ----------
      df_data : pd.DataFrame
          The dataset containing values for all the required variables

      Returns
      -------
      pd.DataFrame
        Returns a dataframe with the granularity,date,hyperparameters,window and performance metrics
    """
    try:
        # broadcast_granularity
        broadcast_gran = broadcast_granularity.value
        global_broadcast_gran = global_broadcast_granularity.value
        df_data['ts_identifier'] = df_data[broadcast_gran].astype(str).apply("_".join, axis=1)
        df_data = df_data.sort_values(by=['ts_identifier','ds'],ascending = True)
        df_data.index = df_data['ds']
        prediction_length = broadcast_test_periods.value
        
        corr_var = corr_config_broadcast.value
        regressors = list(set(corr_var["feat_dynamic_real"]))

        train = pd.DataFrame()
        test = pd.DataFrame()
        ts_comb = df_data['ts_identifier'].unique()
        for comb in ts_comb:
            temp_data = df_data[df_data['ts_identifier']==comb]

            # Train - test split
            train_index_start = temp_data["train_index_start"].iloc[0]
            train_index_end = temp_data["train_index_end"].iloc[0]
            test_i = temp_data["test_index_end"].iloc[0]
            train = pd.concat([train,temp_data.iloc[train_index_start:train_index_end]],ignore_index = False)
            test = pd.concat([test,temp_data.iloc[train_index_start:test_i]],ignore_index = False)

        train_dataset = PandasDataset.from_long_dataframe(
            train,
            item_id="ts_identifier",
            feat_dynamic_real=regressors,
            target = 'y'
        )
        test_dataset = PandasDataset.from_long_dataframe(
            test,
            item_id="ts_identifier",
            feat_dynamic_real=regressors,
            target = 'y'
        )
        freq = train_dataset.freq

        # Updating the default arguments with the parameters provided in the config
        hp_config = broadcast_hyper_parameters.value

        def_args = get_default_args(Trainer)
        for x in hp_config:
            if(x in def_args.keys()):
                temp_val = df_data[x].iloc[0]
                if(type(temp_val)==str):
                    if('[' in temp_val):
                        temp_val = eval(temp_val)
                def_args[x] = temp_val

        def_args_init = get_default_args(DeepStateEstimator)
        for x in hp_config:
            if(x in def_args_init.keys()):
                temp_val = df_data[x].iloc[0]
                if(type(temp_val)==str):
                    if('[' in temp_val):
                        temp_val = eval(temp_val)
                def_args_init[x] = temp_val

        def_args_init['trainer'] = Trainer(**def_args)
        def_args_init['prediction_length'] = prediction_length
        def_args_init['freq'] = freq
        def_args_init['use_feat_static_cat'] = False
        def_args_init['cardinality'] = [1]
        estimator = DeepStateEstimator(**def_args_init)

        mx.random.seed(7)
        np.random.seed(7)
        predictor = estimator.train(train_dataset)

        forecast_it,_ = make_evaluation_predictions(dataset=test_dataset, predictor=predictor)
        forecasts = list(forecast_it)
        reslts_df_list=[]

        for ts_forecasts in forecasts:
            df_temp= pd.DataFrame({"ds" : pd.date_range(ts_forecasts.start_date.to_timestamp(),\
                                                                 periods=prediction_length, freq=freq),
                                'yhat' : ts_forecasts.samples.mean(axis=0),
                                'ts_identifier' : ts_forecasts.item_id},
                                )
            reslts_df_list.append(df_temp)

        df_predictions = pd.concat(reslts_df_list)

        # Sales or Quantity can't be negative hence
        df_predictions["yhat"] = np.where(df_predictions["yhat"]<0,0,df_predictions["yhat"])
        gran = list(set(broadcast_gran+global_broadcast_gran))
        results_pd = test[gran+['ts_identifier','window','ds','y']].reset_index(drop = True)
        results_pd=pd.merge(results_pd,df_predictions,on=['ts_identifier','ds'],how='right')

        # Eval. metrics calculation
        # to handle erroneous results epsilon is set to 1.
        epsilon = 1
        temp_data1 = pd.DataFrame(index= range(1))
        temp_data2 = pd.DataFrame()
        for comb in ts_comb:
            temp_data = results_pd[results_pd['ts_identifier']==comb]
            y_pred = temp_data['yhat']
            y_true = temp_data['y']
            temp_data1['ts_identifier'] = comb
            # Eval. metrics calculation
            temp_data1['mape'] = np.mean(np.abs(y_true - y_pred) / np.maximum(np.abs(y_true), epsilon))*100  
            temp_data1['wmape'] = np.sum(np.abs(y_true - y_pred)) / np.maximum(np.sum(np.abs(y_true)),epsilon)*100  
            temp_data1['bias'] = np.mean((y_true - y_pred))  
            temp_data1['tracking_signal'] = np.sum((y_true - y_pred)) / np.mean(np.abs(y_true - y_pred))
            temp_data1['mae'] = mean_absolute_error(y_true, y_pred)
            temp_data1['rmse']=np.sqrt(mean_squared_error(y_true, y_pred))
            temp_data2 = pd.concat([temp_data2,temp_data1],ignore_index = True)
        
        # Aggregating the metrics based on sales
        sales_col = broadcast_sales.value
        df_data[sales_col] = df_data[sales_col].fillna(0)
        sales_data = df_data.groupby("ts_identifier").agg(Sales= (sales_col,sum)).reset_index()
        sales_sum = sales_data['Sales'].sum()
        sales_data['Sales_per'] = sales_data['Sales']/sales_sum
        metrics_data = pd.merge(temp_data2,sales_data,on = "ts_identifier",how ='left')
        
        results_pd['mape'] = (metrics_data['mape']*metrics_data['Sales_per']).sum()
        results_pd['wmape'] = (metrics_data['wmape']*metrics_data['Sales_per']).sum()
        results_pd['bias'] = (metrics_data['bias']*metrics_data['Sales_per']).sum()
        results_pd['tracking_signal'] = (metrics_data['tracking_signal']*metrics_data['Sales_per']).sum()
        results_pd['mae'] = (metrics_data['mae']*metrics_data['Sales_per']).sum()
        results_pd['rmse'] = (metrics_data['rmse']*metrics_data['Sales_per']).sum()
        
        # To adhere to defined schema
        for x in gran+['window']:   
            results_pd[x] = results_pd[x].astype(str)

        # Append Hyperparameters used
        for x in hp_config:
            results_pd[x] = df_data[x].iloc[0]
        
        del(results_pd['ts_identifier'])
        results_pd['status'] = 'success'
        return results_pd
    
    except Exception as e:
        gran = list(set(broadcast_granularity.value+global_broadcast_granularity.value))
        results_pd = pd.DataFrame(columns = [['ds', 'y', 'yhat','mape','wmape','bias',\
                                            'tracking_signal','mae','rmse','window']+\
                        list(broadcast_hyper_parameters.value.keys()) + ['status'] + gran],index = range(1))
        results_pd[gran] = df_data[gran].head(1).reset_index(drop = True)
        for x in gran:
            results_pd[x] = results_pd[x].astype(str)
        results_pd['status'] = str(e)
        return results_pd

#### Hyperparameter grid
The following function creates a cross product of all the hyperparameters provided in the config file and returns a hyperparamter grid

In [ ]:
# Creating a hyperparameter grid using the set of provided hyperparameters in the config file
def create_hyperparam_space(hp_space: dict) -> pd.DataFrame:
    """Function to create a hyperparameter grid using the set of provided hyperparameters in the config file to be used for Hyperparameter tuning

    Parameters
    ----------
    hp_space : dict
        The set of provided hyperparameters in the config file

    Returns
    -------
    pd.DataFrame
        Returns a hyperparameter grid created from the set of provided hyperparameters
    """    
    df_list = list()
    for x in hp_space:
      df_list.append(pd.DataFrame({x:hp_space[x]}))

    space=df_list[0]

    for x in df_list[1:]:
        space = space.merge(x,how="cross")

    return space

#### Backtesting parallelized using UDF
Based on the Backtesting algorithm (sliding_window/expanding_window), training percentage & test periods specified in the config file, Hyperparameter tuning is performed

In [ ]:
def hyperparameter_tuning(
    algorithm: str,
    train_data: pd.DataFrame,
    hyperparam_space: dict,
    modeling_granularity_conf: list,
    global_modeling_granularity_conf: list,
    train_percentage: float,
    backtesting_test_periods: int,
    test_periods: int,
    stride: int,
    modelling_func: pd.DataFrame = get_prediction_UDF,
) -> pd.DataFrame:
    
    """Function to perform the hyperparameter tuning

    Parameters
    ----------
    algorithm : str
        the algorithm with which the hyperparameter tuning is to be performed(sliding_window/expanding_window)
    train_data : pd.DataFrame
        the modelling dataset
    hyperparam_space : dict
        hyperparameter grid which was created using the set of hyperparameter from config file
    result_schema : StructType
        the schema of all the columns in the dataset
    modeling_granularity_conf : list
        The list of the granularity variables at which the models will be built
    train_percentage : float
        The percentage of data points for the training
    backtesting_test_periods : int
        number of time periods to score in each iteration of backtesting
    test_periods : int
        number of time periods to score in final model building
    stride : int
        number of time periods to stride in each iteration of Backtesting
    modelling_func : pd.DataFrame, optional
        the function name to perform backtesting for the given dataset, by default get_prediction_UDF

    Returns
    -------
    pd.DataFrame
        Returns the dataframe containing the hyperparameter set of modelling granularity x hyperparameter set x window level
    """
    if algorithm == "expanding_window":
        gbcp = list(global_modeling_granularity_conf) + list(hyperparam_space.columns) + ['window']
        unique_skuXds = train_data[modeling_granularity_conf+["#total_weeks"]].drop_duplicates().reset_index(drop = True)
        
        final_list = []
        gran_len = len(modeling_granularity_conf)
        for row1 in range(0,len(unique_skuXds)): 
            Total_weeks = unique_skuXds.loc[row1,'#total_weeks']
            train_interval = int((Total_weeks-test_periods) * train_percentage)
            j = 0
            train_period_ends = Total_weeks-test_periods
            for train_i in range(train_interval,train_period_ends,stride):
                if(train_i+backtesting_test_periods <=train_period_ends):
                    test_i = train_i+backtesting_test_periods
                    final_list.append([unique_skuXds.iloc[row1,index] for index in range(gran_len)] + [0,train_i,train_i+backtesting_test_periods,j+1])
                    j += 1  
  
        # create all windows combination.
        df_windows = pd.DataFrame([tuple(x) for x in final_list],columns =modeling_granularity_conf+\
                                  ['train_index_start','train_index_end','test_index_end','window'])
        f_df = train_data.merge(df_windows,on=modeling_granularity_conf,how="left")
        f_df['temppp'] = 1
        hyperparam_space['temppp'] = 1
        f_df = f_df.merge(hyperparam_space,on='temppp',how="left")
        f_df['gran_tempp'] = f_df[gbcp].astype(str).sum(axis=1)
        unique_pdts = f_df['gran_tempp'].unique()
        new_results = pd.DataFrame()
        for pdt in unique_pdts:
            new_results = pd.concat([new_results,modelling_func(f_df[f_df['gran_tempp']==pdt])])
        return new_results

    
    elif algorithm == "sliding_window":
        gbcp = list(global_modeling_granularity_conf) + list(hyperparam_space.columns) + ['window']
        unique_skuXds = train_data[modeling_granularity_conf+["#total_weeks"]].drop_duplicates().reset_index(drop = True)
        
        final_list = []
        gran_len = len(modeling_granularity_conf)
        for row1 in range(0,len(unique_skuXds)): 
            Total_weeks = unique_skuXds.loc[row1,'#total_weeks']
            train_interval = int((Total_weeks-test_periods) * train_percentage)
            j = 0
            train_period_ends = Total_weeks-test_periods
            train_index_start = 0
            for train_i in range(train_interval,train_period_ends,stride):
                if(train_i+backtesting_test_periods <=train_period_ends):
                    test_i = train_i+backtesting_test_periods
                    final_list.append([unique_skuXds.iloc[row1,index] for index in range(gran_len)] + \
                                      [train_index_start,train_i,train_i+backtesting_test_periods,j+1])
                    j += 1
                    train_index_start = train_index_start+stride
                    
        # create all windows combination.
        df_windows = pd.DataFrame([tuple(x) for x in final_list],columns =modeling_granularity_conf+\
                                  ['train_index_start','train_index_end','test_index_end','window'])
        f_df = train_data.merge(df_windows,on=modeling_granularity_conf,how="left")
        f_df['temppp'] = 1
        hyperparam_space['temppp'] = 1
        f_df = f_df.merge(hyperparam_space,on='temppp',how="left")
        f_df['gran_tempp'] = f_df[gbcp].astype(str).sum(axis=1)
        unique_pdts = f_df['gran_tempp'].unique()
        new_results = pd.DataFrame()
        for pdt in unique_pdts:
            new_results = pd.concat([new_results,modelling_func(f_df[f_df['gran_tempp']==pdt])])
        return new_results

#### Loading the latest Missing_value_treatment file
##### Please update the reading path with the required data path if "Missing value treatment" was not run

In [ ]:
# Reading the latest input file based on timestamp
all_files = [file for file in os.listdir(app_config['output_dir_path']+"/Data_Processing/Missing_value_treatment")]
missing_op_files = [file for file in all_files if "Missing_value_treatment_results (" in file]
missing_op_files = [file.replace(".csv","") for file in missing_op_files]
version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in missing_op_files]
max_date = max(version_dates)
max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
req_file_name = [x for x in missing_op_files if max_date in x]
missing_op_file_path = os.path.join(app_config['output_dir_path']+"/Data_Processing/Missing_value_treatment",req_file_name[0] + ".csv")
# print(missing_op_file_path)

# Reading the data
df = pd.read_csv(missing_op_file_path)
# print(df.shape)

df.rename(columns = {ds_config:"ds", dv_config:"y"}, inplace = True)
logger.info("Data loaded")

df['ds'] = pd.to_datetime(df['ds'])
df[list(set(modeling_granularity_conf+global_modeling_granularity_conf))] = df[list(set(modeling_granularity_conf+global_modeling_granularity_conf))].astype(str)

# Getting the total number of weeks for each time series
temp_df = df.groupby(modeling_granularity_conf).agg({'ds':'count'}).rename(columns={'ds': '#total_weeks'}).reset_index()
df = df.merge(temp_df, on = modeling_granularity_conf ,how = "left")

# 2. Create the hyperparameter space
hpspace = create_hyperparam_space(hyperparameters_conf)
logger.info("Created hyperparameter grid")

# 4.  HP tuning based on algorithm of choice
df_f = hyperparameter_tuning(app_config['validation']['backtesting']['algorithm'],\
                             df,hpspace,modeling_granularity_conf,global_modeling_granularity_conf,\
                             app_config['validation']['train_percentage'],\
                             app_config['validation']['no_of_backtesting_test_periods'],\
                             app_config['validation']['no_of_test_periods'],\
                             app_config['validation']['backtesting']['stride'],modelling_func = get_prediction_UDF)

df_f.to_csv(algo_path + "/Backtesting_results_window_level (" + datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+").csv", index = False)
logger.info("Completed Backtesting")

(156, 91)
<command-3460216454770954>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 df_data['ts_identifier'] = df_data[broadcast_gran].astype(str).apply("_".join, axis=1)
 0%| | 0/50 [00:00<?, ?it/s] 2%|▏ | 1/50 [00:42<34:26, 42.18s/it, epoch=1/1, avg_epoch_loss=11.3] 10%|█ | 5/50 [00:54<06:44, 9.00s/it, epoch=1/1, avg_epoch_loss=9.8]  18%|█▊ | 9/50 [01:05<03:44, 5.48s/it, epoch=1/1, avg_epoch_loss=8.74] 26%|██▌ | 13/50 [01:18<02:47, 4.54s/it, epoch=1/1, avg_epoch_loss=7.93] 36%|███▌ | 18/50 [01:29<01:53, 3.53s/it, epoch=1/1, avg_epoch_loss=7.21] 36%|███▌ | 18/50 [01:40<01:53, 3.53s/it, epoch=1/1, avg_epoch_loss=6.78] 46%|████▌ | 23/50 [01:41<01:22, 3.05s/it, epoch=1/1, avg_epoch_loss=6.69] 56%|█████▌ | 28/50 [01:52<01:01, 2.78s/it, epoch=1/1, avg_epoch_loss=6.31] 66%|██████▌ | 33/50 [02:04<00:44, 2.65s/it, epoch=1/1, avg_epoch_loss=6.01] 76%|███████▌ | 38/50 [02:16<00:30, 2.54s/it, epoch=1/1, avg_epoch_loss=5.79] 86%|████████▌ | 43/50 [02:27<00:17, 2.46s/it, epoch=1/1, avg_epoch_loss=5.61] 96%|█████████▌| 48/50 [02:39<00:04, 2.42s/it, epoch=1/1, avg_epoch_loss=5.46]100%|██████████| 50/50 [02:44<00:00, 3.28s/it, epoch=1/1, avg_epoch_loss=5.41]
<command-3460216454770954>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 df_data['ts_identifier'] = df_data[broadcast_gran].astype(str).apply("_".join, axis=1)
 0%| | 0/50 [00:00<?, ?it/s] 2%|▏ | 1/50 [00:15<12:22, 15.16s/it, epoch=1/1, avg_epoch_loss=11.6] 10%|█ | 5/50 [00:31<04:13, 5.64s/it, epoch=1/1, avg_epoch_loss=10]  20%|██ | 10/50 [00:43<02:28, 3.72s/it, epoch=1/1, avg_epoch_loss=8.65] 20%|██ | 10/50 [00:54<02:28, 3.72s/it, epoch=1/1, avg_epoch_loss=7.85] 30%|███ | 15/50 [00:54<01:47, 3.06s/it, epoch=1/1, avg_epoch_loss=7.69] 40%|████ | 20/50 [01:06<01:22, 2.73s/it, epoch=1/1, avg_epoch_loss=7.04] 50%|█████ | 25/50 [01:23<01:15, 3.03s/it, epoch=1/1, avg_epoch_loss=6.57] 50%|█████ | 25/50 [01:34<01:15, 3.03s/it, epoch=1/1, avg_epoch_loss=6.41] 56%|█████▌ | 28/50 [01:37<01:13, 3.36s/it, epoch=1/1, avg_epoch_loss=6.34] 64%|██████▍ | 32/50 [01:48<00:57, 3.18s/it, epoch=1/1, avg_epoch_loss=6.09] 72%|███████▏ | 36/50 [02:00<00:44, 3.19s/it, epoch=1/1, avg_epoch_loss=5.89] 80%|████████ | 40/50 [02:14<00:32, 3.21s/it, epoch=1/1, avg_epoch_loss=5.73] 80%|████████ | 40/50 [02:24<00:32, 3.21s/it, epoch=1/1, avg_epoch_loss=5.62] 88%|████████▊ | 44/50 [02:30<00:20, 3.49s/it, epoch=1/1, avg_epoch_loss=5.59] 94%|█████████▍| 47/50 [02:40<00:10, 3.47s/it, epoch=1/1, avg_epoch_loss=5.5] 100%|██████████| 50/50 [02:48<00:00, 3.36s/it, epoch=1/1, avg_epoch_loss=5.42]
<command-3460216454770954>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 df_data['ts_identifier'] = df_data[broadcast_gran].astype(str).apply("_".join, axis=1)
 0%| | 0/50 [00:00<?, ?it/s] 2%|▏ | 1/50 [00:24<20:10, 24.71s/it, epoch=1/1, avg_epoch_loss=12] 10%|█ | 5/50 [00:42<05:38, 7.52s/it, epoch=1/1, avg_epoch_loss=10.3] 16%|█▌ | 8/50 [00:53<03:58, 5.67s/it, epoch=1/1, avg_epoch_loss=9.39] 16%|█▌ | 8/50 [01:04<03:58, 5.67s/it, epoch=1/1, avg_epoch_loss=8.63] 24%|██▍ | 12/50 [01:07<02:53, 4.57s/it, epoch=1/1, avg_epoch_loss=8.41] 34%|███▍ | 17/50 [01:21<02:05, 3.80s/it, epoch=1/1, avg_epoch_loss=7.55] 44%|████▍ | 22/50 [01:33<01:30, 3.24s/it, epoch=1/1, avg_epoch_loss=6.95

### Choosing the best hyperparameters

In [ ]:
# Reading the latest file based on timestamp
all_files = [file for file in os.listdir(algo_path)]
backtesting_files = [file for file in all_files if "Backtesting_results_window_level (" in file]
backtesting_files = [file.replace(".csv","") for file in backtesting_files]
version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in backtesting_files]
max_date = max(version_dates)
max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
req_file_name = [x for x in backtesting_files if max_date in x]
backtesting_results_file_path = os.path.join(algo_path,req_file_name[0] + ".csv")
# print(backtesting_results_file_path)

In [ ]:
# Reading the results of backtesting
df = pd.read_csv(backtesting_results_file_path)
df = df[df['status'] == 'success']
# display(df)

In [ ]:
# getting the products count across windows
temp_df = df[list(set(global_modeling_granularity_conf + modeling_granularity_conf)) + ["window"]].drop_duplicates()
temp_df1 = temp_df.groupby(global_modeling_granularity_conf+["window"]).agg(pdts_count = (modeling_granularity_conf[0], 'count')).reset_index()
pdts_sum = temp_df1.groupby(global_modeling_granularity_conf).agg(overall_pdts = ("pdts_count", 'sum')).reset_index()
temp_df1 = temp_df1.merge(pdts_sum,how='left',on=global_modeling_granularity_conf)
temp_df1['pdts_perc'] = temp_df1['pdts_count'] / temp_df1['overall_pdts']
# display(temp_df1)

In [ ]:
# Roll up the metrics at Modeling granularity x Hyperparameter space x window level
window_level = list(hyperparameters_conf.keys()) + global_modeling_granularity_conf + ["window"]
# print(window_level)
window_level_results = df.groupby(window_level)[["mape","wmape","bias","tracking_signal","mae","rmse"]].min().reset_index()

# getting the products_percentage
window_level_results = window_level_results.merge(temp_df1[global_modeling_granularity_conf + ['window','pdts_perc']],\
                                                 on=global_modeling_granularity_conf+['window'],how="left")
# display(window_level_results)

In [ ]:
# Roll up the metrics at Modeling granularity x Hyperparameter space level
all_metrics=["mape","wmape","bias","tracking_signal","mae","rmse"]
# window_level_results[all_metrics] = window_level_results[all_metrics] * window_level_results['pdts_perc']

window_level_results[all_metrics] = window_level_results[all_metrics].multiply(window_level_results['pdts_perc'], axis="index")


hyperparam_level = global_modeling_granularity_conf + list(hyperparameters_conf.keys())
hyperparam_level_results = window_level_results.groupby(hyperparam_level)[["mape","wmape","bias","tracking_signal","mae","rmse"]].sum().reset_index()
# display(hyperparam_level_results)

In [ ]:
hyperparam_level_results.to_csv(algo_path + "/Backtesting_results_hyperparameter_level (" + datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+").csv", index = False)
logger.info("Exported backtesting results")

INFO:custom_log:Exported backtesting results

#### Pandas UDF for getting best hyperparameters & MLFlow tracking

In [ ]:
hyperparam_level_results['algorithm'] = 'DeepState'
hyperparam_level_results['result_type'] = 'backtesting'

In [ ]:
def get_best_hyperparameters(final_hyperparam_df: pd.DataFrame) -> pd.DataFrame:
    """Function for getting the best hyperparameters results for each modeling granularity along with MLFlow tracking using the broadcasted information from the config file

    Parameters
    ----------
    final_hyperparam_df : pd.DataFrame
        Dataframe containing the results of Backtesting 

    Returns
    -------
    pd.DataFrame
        Dataset with best hyperparameter set for each modeling granularity
    """
    try:
        results_pd = {}

        # Get the modeling granularity
        global_broadcast_gran = global_broadcast_granularity.value
        # Get the experiment id
        tracking_value = broadcast_tracking.value.copy()
        df_len = final_hyperparam_df.count()
        
        # To adhere to defined schema
        for x in global_broadcast_gran:
            results_pd[x] = final_hyperparam_df[x].astype(str).iloc[0]

        hp_config = broadcast_hyper_parameters.value
        
        if tracking_value["tracking_needed"] == True:
            if tracking_value["type"] != "Managed":
                if tracking_value["tracking_uri"] is not None:
                    mlflow.set_tracking_uri("file:" + tracking_value["tracking_uri"])
                    experiment_id = mlflow.set_experiment(tracking_value["mlflow_experiment_id"])
                    tracking_value['mlflow_experiment_id'] = experiment_id.experiment_id
                    
            # Add MLFlow code here
            with mlflow.start_run(experiment_id=tracking_value["mlflow_experiment_id"]):
                mlflow.log_param("algorithm", "DeepState")
                mlflow.log_param("result_type", "backtesting")
                for x in global_broadcast_gran:
                    mlflow.log_param(x, final_hyperparam_df[x].iloc[0])
                for row in range(0, len(final_hyperparam_df)):
                    with mlflow.start_run(experiment_id=tracking_value["mlflow_experiment_id"], nested=True):
                        for x in hp_config:
                            mlflow.log_param(x, final_hyperparam_df[x].iloc[row])
                        for x in ["mape","wmape","bias","tracking_signal","mae","rmse"]:
                            mlflow.log_metric(x, final_hyperparam_df[x].iloc[row])

        metric = broadcast_metric.value

        # Sort using the metric of interest
        if metric in ["wmape", "mape", "mad", "mae", "rmse"]:
            final_hyperparam_df = final_hyperparam_df.sort_values(
                metric, ascending=True
            )
        elif metric in ["tracking_signal","bias"]:
            final_hyperparam_df.sort_values(metric, ascending=True,key=abs, inplace=True)
        else:
            final_hyperparam_df.sort_values(metric, ascending=True, inplace=True)

        # Adding the best hyperparameter and related metrics
        for x in hp_config:
            results_pd[x] = final_hyperparam_df[x].iloc[0]

        for x in ["mape", "wmape", "bias", "tracking_signal", "mae", "rmse"]:
            results_pd[x] = final_hyperparam_df[x].iloc[0]

        results_pd["status"] = "success"
        return pd.DataFrame.from_dict([results_pd])

    except Exception as e:
        gran = list(set(global_broadcast_granularity.value))
        results_pd = pd.DataFrame(
            columns=[["mape", "wmape", "bias", "tracking_signal", "mae", "rmse"]+ list(broadcast_hyper_parameters.value.keys())+ \
                     ["status"]+ gran],index=range(1))
        results_pd[gran] = final_hyperparam_df[gran].head(1).reset_index(drop=True)
        for x in gran:
            results_pd[x] = results_pd[x].astype(str)
        results_pd["status"] = str(e)
        return results_pd

In [ ]:
# Get the best hyperparameter combination for each modeling granularity
hyperparam_level_results['gran_tempp'] = hyperparam_level_results[global_modeling_granularity_conf].astype(str).sum(axis=1)
unique_pdts = hyperparam_level_results['gran_tempp'].unique()
best_hyperparam_results = pd.DataFrame()
for pdt in unique_pdts:
    best_hyperparam_results = pd.concat([best_hyperparam_results,get_best_hyperparameters(hyperparam_level_results[hyperparam_level_results['gran_tempp']==pdt])])

#### Writing the best hyperparameter results

In [ ]:
best_hyperparam_results.to_csv(algo_path + "/Best_hyperparameters (" + datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+").csv", index = False)
logger.info("Exported best hyperparameter results")

INFO:custom_log:Exported best hyperparameter results

In [ ]:
# Exporting config file
config_file_name = "config_for_exp_id_"+str(broadcast_tracking.value['mlflow_experiment_id']) + " (" +datetime.today().strftime('%Y-%m-%d-%H-%M-%S-%f')[:-3]+").yml"
config_path1 = os.path.join(config_path,config_file_name)
with open(config_path1, 'w') as file:
    yaml.dump(temp_config, file, default_flow_style=False,sort_keys=False)

In [ ]:
# Move from tmp directory to req. location in datalake
import platform
plat_sys = platform.system()

if(plat_sys!='Windows'):
    log_file = log_file.replace(' (', '\ \(').replace(')','\)')
    os.system('mv /tmp/{0} {1}'.format(log_file,logs_path))